In [10]:
import numpy as np
import pandas as pd
import sys
sys.path.append("..")
import analysis as an
from utilsGDP import config, io, models
from analysis import selector as se

In [11]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

this will be in the select code, data adapting for the multicountry analysis

In [4]:
from analysis import reader as re

In [5]:
from analysis import selectorall as sall

In [6]:
from analysis import countrylist as clist

In [7]:
ncorr=50

In [9]:
indicators=sall.selection(ncorr)

In [12]:
indicators

IndicatorCode
SI.SPR.PCAP.ZG              1.000000
NextYearGDP                 1.000000
SI.SPR.PC40.ZG              1.000000
SI.SPR.PC40.05              0.720355
SI.SPR.PCAP.05              0.714283
LP.LPI.LOGS.XQ              0.403281
LP.LPI.TRAC.XQ              0.402993
LP.LPI.CUST.XQ              0.392186
LP.LPI.INFR.XQ              0.387387
LP.LPI.OVRL.XQ              0.384166
per_allsp.ben_q1_tot        0.363735
FB.POS.TOTL.P5              0.347135
per_si_allsi.ben_q1_tot     0.339280
NY.GNP.MKTP.KD.ZG           0.332176
LP.LPI.ITRN.XQ              0.318228
LP.LPI.TIME.XQ              0.315624
CM.MKT.INDX.ZG              0.314947
NY.GDP.MKTP.KD.ZG           0.313394
SL.TLF.0714.WK.FE.TM        0.313309
SL.TLF.0714.WK.MA.ZS        0.307164
SL.TLF.0714.SW.MA.ZS        0.307164
NY.GNP.PCAP.KD.ZG           0.304058
SI.SPR.PC40                 0.290013
SL.TLF.0714.SW.FE.ZS        0.288936
SL.TLF.0714.WK.FE.ZS        0.288936
IC.FRM.FEMM.ZS              0.288655
NY.GDP.PCAP.KD.ZG       

In [13]:
predictors,pre2010=sall.indicators(ncorr)
indicators=pd.DataFrame(indicators)
indexs=indicators.index.values.tolist()
predictors=np.nan_to_num(predictors)

Todo esto de arriba llama a funciones selectall para obtener la información del dataset

In [14]:
predictors

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  5.98439075,  0.        , ...,  0.        ,
        -3.13756141,  0.        ],
       [ 0.        , 11.37592123,  0.        , ...,  0.        ,
        31.87712627,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.09645039,  0.        ]])

In [15]:
pre2010=pre2010.fillna(0)
npre2010=pre2010[indexs].to_numpy()

In [16]:
x=predictors[:,:-1]
y=predictors[:,-1]

x son todos los elementos de predictors menos la ultima columna, la ultima es y que es justamente GDPgrowth para el siguiente año

In [17]:
import time

In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

gbm_hyperparams = {
    'n_estimators': 2000,
    'max_depth': 5,
    'learning_rate': 0.1,
    'loss': 'huber',
    'criterion': 'mse',
    'validation_fraction': 0.02,
    'n_iter_no_change': 20
}
gbm_model = GradientBoostingRegressor(**gbm_hyperparams)
t0 = time.time()
gbm_model.fit(X_train, y_train)
print(f"Elapsed time training: {time.time() - t0} seconds")
t0 = time.time()
gbm_y_pred = gbm_model.predict(X_test)
print(f"Elapsed time predicting: {time.time() - t0} seconds")


Elapsed time training: 24.830583095550537 seconds
Elapsed time predicting: 0.03490161895751953 seconds


In [19]:
print(f"RMSE: {mean_squared_error(y_test, gbm_y_pred)**0.5}")
print(f"R^2: {r2_score(y_test, gbm_y_pred)}")

results_df = X_test.copy()
results_df["y_real"] = y_test
results_df["y_pred"] = gbm_y_pred.astype(float)
results_df["err"] = results_df["y_real"] - results_df["y_pred"]
results_df["%_err"] = results_df["err"] / results_df["y_real"] * 100
results_df

RMSE: 2.57828298724989
R^2: 0.12910517378730624


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [20]:
prediction=gbm_model.predict(npre2010[:,:-1]).astype(float)

In [23]:
prediction30=gbm_model.predict(npre2010[:,:-1]).astype(float)

In [24]:
prediction30

array([ 7.37476852e-01,  1.90257139e-01,  5.76091865e-01,  7.17208017e-01,
        1.69533465e+00,  7.70023497e-01,  2.97805830e-01,  4.81905699e+00,
        2.90758693e+00,  7.37476852e-01,  1.98721542e+00,  3.41948553e+00,
        3.12964102e+00,  4.91373535e-01,  2.22776941e-01,  5.71195295e+00,
        3.30518783e-01,  5.86685188e-01,  2.51234043e-01,  6.25729957e+00,
        1.76285590e+00,  2.33252294e+00,  3.16377711e+00,  8.91271654e-01,
        3.68749476e-01,  6.69296780e-01,  3.71568612e-01,  2.21818532e+00,
        5.62995231e+00,  1.04372969e+00,  6.78664193e-01,  1.68245616e+00,
        9.74109251e-01,  2.78536812e+00,  5.80207641e+00,  4.61202202e+00,
        7.37476852e-01,  4.10631188e+00,  1.90149609e+00,  1.36649928e-01,
        3.86207887e-01,  6.20601129e-01,  8.95399875e-01,  2.33097657e-01,
        1.99881212e+00,  1.13328840e+00,  1.60414638e+00,  5.25095244e+00,
        7.37476852e-01,  7.37476852e-01,  2.43081540e+00,  6.71542275e+00,
        1.60421459e+00,  

In [21]:
prediction

array([ 1.27602799e-01, -2.99306208e-01,  5.48403669e-01,  1.75529863e-01,
        3.63261180e+00,  6.96249401e-02,  1.72992330e-01,  3.91121751e+00,
        3.53429461e+00,  1.27602799e-01,  2.07498649e+00,  3.80649415e+00,
        1.91076488e+00,  3.39973554e-02,  8.66179355e-02,  6.39439647e+00,
        4.06122385e-01,  3.35441138e-01,  1.75675654e+00,  6.58314608e+00,
        1.82201294e+00,  3.16089075e+00,  3.27886315e+00,  1.21277602e-01,
        1.12465990e+00, -8.66482871e-02,  1.83935963e-01,  1.95493777e+00,
        4.98346950e+00,  3.36011260e-01,  1.75232954e-01,  1.65179095e+00,
        4.60439682e-01,  2.49572170e+00,  6.19490941e+00,  4.95320595e+00,
        1.27602799e-01,  6.11106567e+00,  1.97478690e+00, -3.29740345e-02,
       -2.33549693e-01,  1.06356614e-01,  6.72770587e-02,  4.43652461e-01,
        2.14201335e+00,  1.29624658e+00,  1.84432327e+00,  5.71611180e+00,
        1.27602799e-01, -2.97112363e-01,  1.72371554e+00,  7.13210960e+00,
        1.70580289e+00,  

In [27]:
prediction300=gbm_model.predict(npre2010[:,:-1]).astype(float)

In [28]:
prediction300

array([ 6.99393135e-01, -7.31269993e-03,  4.35784088e-01,  5.45677868e-01,
        1.20986321e+00, -3.57126275e-02,  1.35080038e-02,  5.34024258e+00,
        3.78986343e+00,  6.99393135e-01,  2.18867099e+00,  3.89275329e+00,
        2.66975501e+00,  9.84819861e-03,  2.90519743e-04,  6.48058278e+00,
        4.67012623e-01,  3.90491649e-01,  5.09812727e-01,  6.51058637e+00,
        3.94693251e+00,  3.08584089e+00,  3.47102142e+00,  8.50378548e-03,
        1.15008173e+00, -4.23798462e-01, -2.88433924e-02,  2.90386304e+00,
        6.64584398e+00,  1.31178328e-01,  3.03451045e-02,  1.80514614e+00,
        8.86354303e-02,  2.69579155e+00,  6.49648050e+00,  5.03649357e+00,
        6.99393135e-01,  4.58567009e+00,  2.08592550e+00,  4.42249695e-03,
        2.26093594e-01,  4.33924247e-02,  4.65889831e-02,  5.14764456e-02,
        2.09096834e+00,  1.22129789e+00,  2.30353837e+00,  5.92695775e+00,
        6.99393135e-01, -3.56839989e-02,  2.27180691e+00,  7.00685671e+00,
        2.13588122e+00,  

In [37]:
predictiona=gbm_model.predict(npre2010[:,:-1]).astype(float)

In [39]:
predictiona

array([ 7.00363779e-01, -3.95174553e-02,  4.07147626e-01,  4.59511177e-01,
        1.58319632e+00, -1.54134826e-02,  1.51339967e-03,  4.42910745e+00,
        4.98108584e+00,  7.00363779e-01,  2.20638127e+00,  3.88782444e+00,
        2.41606813e+00, -4.47402213e-03, -3.68087924e-02,  6.48292217e+00,
        4.84400901e-01,  3.95644600e-01,  4.57989623e-01,  6.50662009e+00,
        5.12631133e+00,  3.10257832e+00,  3.49378434e+00, -8.93628407e-03,
        1.21394287e+00,  7.99708217e-01, -1.03164860e-02,  3.94681069e+00,
        6.61066961e+00, -3.23310235e-02, -2.44967880e-03,  1.78319641e+00,
        7.02738700e-03,  2.69337180e+00,  6.53900244e+00,  4.97426101e+00,
        7.00363779e-01,  3.40809272e+00,  1.62337475e+00,  1.74318176e-02,
        2.32162080e-01,  1.18067128e-03,  5.38250087e-02,  2.44299297e-01,
        2.13200503e+00,  1.20418584e+00,  2.33005378e+00,  5.90972355e+00,
        7.00363779e-01, -1.84919768e-01,  2.48545668e+00,  7.00182603e+00,
        1.95509924e+00,  

In [25]:
predictionhuber=gbm_model.predict(npre2010[:,:-1]).astype(float)

In [26]:
predictionhuber

array([ 2.60144126e-02,  1.43707448e-02,  4.03882587e-01,  6.56075048e-01,
        1.38421848e+00,  3.77818004e-02,  7.35586138e-04,  3.09883487e+00,
        3.23036134e+00,  2.60144126e-02,  2.18327988e+00,  3.87201441e+00,
        5.03129412e+00, -1.14719814e-03,  2.42738425e-02,  6.45347702e+00,
        4.20995993e-01,  3.58284674e-01,  9.92605152e-01,  6.49974114e+00,
        2.86396067e+00,  3.11947223e+00,  3.51052332e+00, -7.36027660e-03,
        1.19217489e+00,  4.10974487e-01,  4.97467002e-02,  3.46885259e+00,
        6.58539090e+00,  7.12731568e-02,  1.74291965e-02,  1.77380657e+00,
        6.48092505e-02,  2.68084631e+00,  6.48757902e+00,  4.94305053e+00,
        2.60144126e-02,  6.34107750e+00,  1.69107813e+00, -9.13511431e-03,
       -9.43515534e-02,  2.65121861e-02,  9.57161742e-03,  3.08520716e-01,
        2.10464532e+00,  1.20807512e+00,  2.34025877e+00,  5.91402724e+00,
        2.60144126e-02,  1.06705682e-02,  9.95125586e-01,  7.02643695e+00,
        3.02954093e+00,  

In [39]:
clist.numadapt()

,CountryCode,ShortName,num
0,AFG,Afghanistan,1
1,ALB,Albania,2
2,DZA,Algeria,3
3,ASM,American Samoa,4
4,ADO,Andorra,5
...,...,...,...
242,WBG,West Bank and Gaza,243
243,WLD,World,244
244,YEM,Yemen,245
245,ZMB,Zambia,246


In [46]:
columna=clist.numadapt()

In [47]:
columna=columna['CountryCode']

In [48]:
predictiondf=pd.DataFrame(prediction, columna)

In [50]:
predictiondf

,0
CountryCode,
AFG,0.127603
ALB,-0.299306
DZA,0.548404
ASM,0.175530
ADO,3.632612
...,...
WBG,0.680442
WLD,-0.107204
YEM,0.124254


In [51]:
predictiondf.to_csv('PREDICTION.csv')